# This is the python template for Assignment 04.  
- You must use this template.  
- You must not change any signatures of the methods, only edit the sections indicated with "Write your code here."  
- The return of every function has to be in the right format, otherwise this is a desk reject.  
- Plagiarism leads to failing the assignment!  
- We will terminate the script after 10 min, try to use efficient algorithms.

In [35]:
import pandas as pd

In [36]:
def get_name():
    name = "Md Adnan Karim"
    return name
def get_matriculationnumber():
    matric = 7002536
    return matric

## Useful information:

The structure of a CART is a dict. Use the same names as shown in the example, using other names makes your format invalid and leads to a desk reject.

In [37]:
cart = { "name":"X", "mean":456, "split_by_feature": "aes", "error_of_split": 0.0,
        "successor_left": { "name":"XL", "mean":1234, "split_by_feature": None, "error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          },
        "successor_right":{ "name":"XR", "mean":258, "split_by_feature": None,"error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          }
       }

The names of the features must be used as defined in this list, using other names makes your format invalid and leads to a desk reject.

In [38]:
features = ["secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
            "timestamp", "segmentation", "onehundredmb", "onegb"]

# Task 1: Create a CART

In [39]:
def tree_builder(sample_set_csv, node = "X", mean = None):

    while not (sample_set_csv.shape[0] == 0 or len(sample_set_csv.iloc[:,-1].unique()) == 1):
              
    
        l, r, l_sqr, r_sqr, summed_sqr, mean = get_sample_mean(sample_set_csv)
        #print(sum_sqr)
        
        #get min value and key from summed sqr dict
        value = min(summed_sqr.values())
        key = [k for k, v in summed_sqr.items() if v == value]
    
        subset_left = sample_set_csv[sample_set_csv[key[0]] == 1]
        subset_right = sample_set_csv[sample_set_csv[key[0]] == 0]
        successor_left = tree_builder(subset_left, node + "L", mean)
        successor_right = tree_builder(subset_right, node + "R", mean)

        
        return {"name":node, "mean":mean, "split_by_feature": key[0], "error_of_split":value,
                "successor_left": successor_left, "successor_right": successor_right}
    
    return {"name":node, "mean":round(sample_set_csv.iloc[:, -1].mean(), 2), "split_by_feature": None, "error_of_split":None,
            "successor_left": None, "successor_right": None}


#print(feat, last_col, first_col)
def get_sample_mean(x):
    feat=x.columns
    feat = list(feat)
    last_col=feat[-1]
    first_col=feat[0]
    feat.remove(last_col)
    feat.remove(first_col)
    l_mean={}
    r_mean={}
    l_sqr={}
    r_sqr={}
    summed_sqr={}

    mean = round(sum(x.iloc[:, -1])/len(x.iloc[:, -1]), 2)

    for i in feat:
        if x[i].eq(1).all():
            continue
        elif x[i].eq(0).all():
            continue
        else:
            #calculate L and R
            l_mean[i] = sum(x.loc[x[i] == 1, last_col])/x[i].value_counts()[1]
            r_mean[i] = sum(x.loc[x[i] == 0, last_col])/x[i].value_counts()[0]
            
            #calculate square loss L and R
        for key, value in l_mean.items():
            if key==i:
                l_sqr[i]=sum((x.loc[x[i] == 1, last_col]-value)**2)

        for key, value in r_mean.items():
            if key==i:
                r_sqr[i]=sum((x.loc[x[i] == 0, last_col]-value)**2)

    for key in l_sqr:
        if key in r_sqr:
            summed_sqr[key]=round(l_sqr[key] + r_sqr[key], 2)
    
    summed_sqr=dict(sorted(summed_sqr.items()))
    return [l_mean, r_mean, l_sqr, r_sqr,summed_sqr,mean]

In [40]:
def get_cart(sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    # TODO: Write your code here. And change the return.
    func = tree_builder(df)
    #print(func)
    return func

# Task 2a: Highest influencing feature

In [41]:
# Write your helper functions here, if needed

In [42]:
def get_highest_influence_feature(cart):
    # TODO: Write your code here. And change the return.
    return cart.get("split_by_feature")

# Task 2b: Calculate the error rate

In [44]:
# Write your helper functions here, if needed
def get_tree_mean(cart, lst):
    for i in cart.keys():
        if i=="mean":
            if cart.get(i)!=None:
                lst.append(cart.get(i))
        elif i=="successor_left" and isinstance(cart.get(i), dict):
            
            get_tree_mean(cart.get(i),lst)
        elif i == "successor_right" and isinstance(cart.get(i), dict):
            
            get_tree_mean(cart.get(i),lst)

In [45]:
def get_error_rate(cart, sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    # TODO: Write your code here. And change the return.
    
    tree_actual=[]
    get_tree_mean(get_cart(sample_set_csv),tree_actual)
    tree_predicted=[]
    get_tree_mean(cart,tree_predicted)

    tree_diff = list(abs(tree_predicted[i]-tree_actual[i]) for i in range(0,len(tree_predicted)))

    return sum(tree_diff)/(len(tree_diff))

# Task 2c: Generate optimal configuration

In [46]:
# Write your helper functions here, if needed


In [47]:
def get_optimal_configuration(cart, partial_configuration):


    # TODO: Write your code here. And change the return.
    for i in cart.keys():

        if i=="split_by_feature":
            if cart.get(i)!=None:
                if i not in partial_configuration:
                    if (cart.get("successor_left")!=None and isinstance(cart.get("successor_left"), dict)) and (cart.get("successor_right")!=None and isinstance(cart.get("successor_right"), dict)):
                        #print(cart.get("successor_left").get("mean"))
                        if cart.get("successor_left").get("mean") < cart.get("successor_right").get("mean"):
                            partial_configuration.add(cart.get(i))
                        elif cart.get("successor_right").get("mean") < cart.get("successor_left").get("mean"):
                            if cart.get(i) == "aes":
                                partial_configuration.add("blowfish")
                            elif cart.get(i) == "blowfish":
                                partial_configuration.add("aes")
                            elif cart.get(i) == "rar":
                                partial_configuration.add("zip")
                            elif cart.get(i) == "zip":
                                partial_configuration.add("rar")
                            elif cart.get(i) == "onehundredmb":
                                partial_configuration.add("onegb")
                            elif cart.get(i) == "onegb":
                                partial_configuration.add("onehundredmb")



        elif i=="successor_left" and isinstance(cart.get(i), dict):
            if cart.get(i)!=None:
                get_optimal_configuration(cart.get(i),partial_configuration)

        elif i == "successor_right" and isinstance(cart.get(i), dict):
            if cart.get(i)!=None:
                get_optimal_configuration(cart.get(i),partial_configuration)
    

    partial_configuration.add("algorithm")
    partial_configuration.add("secompress")
    return partial_configuration

# Tests:  
In the following cells, we provide you some test cases (but not all possible test cases!)

In [48]:
# Task 1

test_cart = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'segmentation', 'error_of_split': 6.0, 
             'successor_left': 
                 {'name': 'XL', 'mean': 772.0, 'split_by_feature': 'onegb', 'error_of_split': 0.0, 
                  'successor_left': 
                      {'name': 'XLL', 'mean': 770.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }, 
                  'successor_right': 
                      {'name': 'XLR', 'mean': 773.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }
                 }, 
             'successor_right': 
                 {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                  'successor_left': None, 
                  'successor_right': None}
            }


if get_cart("Performance_01.csv") == test_cart:
    print("passed")
else:
    print("failed")

passed


In [49]:
# Task 2b
if get_error_rate(test_cart, "Performance_02b.csv") == 5:
    print("passed")
else:
    print("failed")

passed


In [50]:
# Task 2c
test_cart_v2 = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'zip', 'error_of_split': 0.0, 
                 'successor_left': {'name': 'XL', 'mean': 772.0, 'split_by_feature': None, 'error_of_split': None, 
                                    'successor_left': None, 
                                    'successor_right': None}, 
                 'successor_right': {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                                     'successor_left': None, 
                                     'successor_right': None}
                }

optimal_config = get_optimal_configuration(test_cart_v2, {"secompress", "encryption", "aes", "algorithm", "signature",
                                                        "timestamp", "segmentation", "onehundredmb"})
reference = {'aes', 'algorithm', 'encryption', 'onehundredmb', 'rar', 'secompress', 'segmentation', 'signature',
            'timestamp'}

#print(optimal_config)
if optimal_config == reference:
    print("passed")
else:
    print("failed")

passed
